In [2]:
import pandas as pd
import numpy as np
from datetime  import datetime  
from datetime import timedelta  
import lightgbm as lgb
from sklearn import preprocessing, metrics
from sklearn.model_selection import ParameterGrid
import matplotlib.pyplot as plt
import seaborn as sns
import mlflow
import mlflow.lightgbm
from pandas.plotting import autocorrelation_plot
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf


In [ ]:
data_location = '../00_data/sample_submission.csv'
submission = pd.read_csv(data_location)

In [ ]:
data_location = '../04_submissions/best_submission_day1-5.csv'
best_submission = pd.read_csv(data_location)

In [ ]:
X_train = pd.read_pickle('../01_preprocessed_data/X_1_train.pkl')
y_train = pd.read_pickle('../01_preprocessed_data/y_1_train.pkl')
X_val = pd.read_pickle('../01_preprocessed_data/X_val_pred_lgbm.pkl')
y_val = pd.read_pickle('../01_preprocessed_data/y_1_val.pkl')
X_test = pd.read_pickle('../01_preprocessed_data/X_1_test.pkl')

In [ ]:
X_train.groupby(['demand_type'])['demand_type', 'demand'].agg(['mean']).reset_index()

In [ ]:
X_train[X_train['id'] == 'FOODS_3_120_CA_3_validation']

In [ ]:
food = X_train.loc[X_train['id'] == 'FOODS_3_120_CA_3_validation', ['date', 'demand']]
food = food[365*2: ]
sns.lineplot(x="date", y="demand", data=food[100:200])

In [ ]:
series = food['demand']
sns.lineplot(x = food.loc[100:200, 'date'], y = series.diff()[100:200])

In [ ]:
series = food['demand']
plot_acf(series.diff().dropna())

In [ ]:
series.diff()

In [ ]:
plot_pacf(series.diff().dropna())

In [ ]:
model = ARIMA(series[1000:, ], order=(9, 1, 2))
model_fit = model.fit(disp=0)
print(model_fit.summary())

In [ ]:
residuals = pd.DataFrame(model_fit.resid)
residuals.plot()
plt.show()
residuals.plot(kind='kde')
plt.show()
print(residuals.describe())

In [ ]:
abs(residuals).sum()

In [ ]:
residuals

In [ ]:
model = pm.auto_arima(food, start_p=1, start_q=1,
                           max_p=3, max_q=3, m=12,
                           start_P=0, seasonal=True,
                           d=1, D=1, trace=True,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=True)


In [ ]:
str(model)

In [ ]:
model.plot_diagnostics(figsize=(7,5))
plt.show()

model.resid()

In [ ]:
abs(model.resid()).sum()

In [ ]:
food = X_val.loc[X_val['id'] == 'FOODS_3_120_CA_3_validation', ['date', 'demand_pred', 'demand']]

In [ ]:
food['arima'] = model_fit.predict(28)

In [ ]:
model_fit.predict(start = 1, end = 28)

In [ ]:
food

In [ ]:
abs(food['demand'] - food['arima']).sum()

In [ ]:
abs(food['demand'] - food['demand_pred']).sum()